# Finding the proportion of irrational length line segments between all points on an integer lattice

Concerning [this question in math.stackexchange](https://math.stackexchange.com/questions/2449336/line-segments-of-irrational-length-on-the-lattice/).

Here I've taken a guess on the number of Pythagorean triples in the $n\times n$ lattice as
$$
\sum_{k=1}^{n-1} 4 (n-k) L(k) 
$$
the rationalle being - for a leg of length $k$, there can be $(n-k)$ places to put it laterally, and then 4 is a sloppy estimation for the rotational symmetry of this problem. There's a lot of overlaps. Evidently it's not going to work.

In [44]:
import numpy as np
import primefac
import math

def count_num_squares(n):
    # This does a brute force count for an n x n lattice
    num_sq = 0
    
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    # The distance between the point (i,j) and (k,l)
                    dist_sq = (k-i)*(k-i) + (l-j)*(l-j)
                    dist = np.sqrt(dist_sq)
                    num_sq += (dist_sq == int(dist)*int(dist))
                    
    return (num_sq - n*n) // 2 # (we subtract n*n because we've counted the singletons... i.e. 0 length)
        
def L_prim(k):
    # Find the number of primitive pythagorean triples that k is a leg of
    pf = primefact.factorint(k)

    if pf[2] % 2 == 0:
        return 0
    
    return len(pf)

def L(k):
    # Find the number of primitive *and* non-primitive pythagorean triples that k could be a member of
    pf = primefac.factorint(k)
    
    res = 1.0
    if pf.get(2,0) > 0:
        res *= (2*pf[2] - 1)
        del pf[2]
        
    for a in pf:
        res *= (2*pf[a] + 1)

    return (res - 1) // 2

def calc_num_pyth_squares(n):
    # This uses the L above to do the counting
    num_sq = 0
    
    for k in range(n):
        num_sq += L(k) * (n - k) * (n-k)
    
    return num_sq

def calc_num_flat(n):
    return n * (n-1) * n

print('n \t Total cnx \t Int-len count \t Int-len calc \t Diags calc \t Straights calc')
for n in range(2,14):
    print('{:d} \t {:d} \t\t {} \t\t {:d} \t\t {:d} \t\t {:d}'.format(n, n*n*(n*n-1)//2, count_num_squares(n), int(calc_num_pyth_squares(n) + calc_num_flat(n)), int(calc_num_pyth_squares(n)),  calc_num_flat(n) ))

n 	 Total cnx 	 Int-len count 	 Int-len calc 	 Diags calc 	 Straights calc
2 	 6 		 4 		 4 		 0 		 4
3 	 36 		 18 		 18 		 0 		 18
4 	 120 		 48 		 49 		 1 		 48
5 	 300 		 108 		 105 		 5 		 100
6 	 630 		 204 		 194 		 14 		 180
7 	 1176 		 342 		 324 		 30 		 294
8 	 2016 		 528 		 503 		 55 		 448
9 	 3240 		 780 		 740 		 92 		 648
10 	 4950 		 1100 		 1045 		 145 		 900
11 	 7260 		 1494 		 1427 		 217 		 1210
12 	 10296 		 1968 		 1894 		 310 		 1584
13 	 14196 		 2576 		 2457 		 429 		 2028


Ideally in the results above we want ```Int-len calc``` to be the same as ```Int-len count``` (stands for integer-length counted). This is made up of the straight-connections that are calculated in the last column (which seems to be correct) and the diagonals in the second last column, where we use the $L(s)$ function [defined here](http://mathworld.wolfram.com/PythagoreanTriple.html) in a heuristic way to approximate the number of Pythagorean triples that would fit in the $n\times n$ lattice. Sometimes this approach gets it right but usually not.